In [1]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd


In [2]:
# get events from n days ago
iso = 562 #Look in ACLED documentation ISO code for number this is for Niger
limit = 400

api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}'.format(iso)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

https://api.acleddata.com/acled/read?terms=accept&iso=562 <class 'str'>


In [3]:
response = requests.get(api_url)
data = response.json()
data.keys()
data['count'] 

500

### From the documentation we know this is the max return --- How can we get all the results?

In [4]:
# Let's mkae a function that updates our search to get the new pages

def ping_acled(api_url): 
    '''
    Takes one parameter search term for API
    '''
    
    response = requests.get(api_url)
    data = response.json()
    return data
    

In [5]:

results = [] # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

while num_results == 500: #if less 500 or 0 we know we have all the results
    print ("starting ", page, " ", num_results) #just to see our progress
    api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'.format(iso,page) #the search
    data = ping_acled(api_url) #call the previous function 
    results.append(data['data']) #store in our results
    count += data['count'] #Track number of results
    num_results = data['count'] #update our condition
    page += 1 #update our page variable
    print ("Total Results ", count) #Track our progress
    



starting  1   500
Total Results  500
starting  2   500
Total Results  1000
starting  3   500
Total Results  1417


In [6]:
#Now I want to put them together into one giant result
super_list = []
for res in results: 
    super_list += res
    print (len(super_list))

500
1000
1417


In [7]:
#convert it into an pandas data frame or just use your data structure and do more stuff
niger_res = pd.DataFrame(super_list)
niger_res.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7239541,562,NIR1469,1469,2020-10-03,2020,1,Riots,Mob violence,Rioters (Niger),...,Diffa,13.3200,12.6100,1,Whatsapp,New media,"On 3 October 2020, twenty-one prisoners (presu...",0,1601932936,NER
1,7239452,562,NIR1467,1467,2020-10-01,2020,1,Protests,Peaceful protest,Protesters (Niger),...,Niamey,13.5200,2.1200,1,Studio Kalangou,National,"On 1 October 2020, a number of teachers staged...",0,1601932936,NER
2,7239043,562,NIR1468,1468,2020-10-01,2020,1,Protests,Peaceful protest,Protesters (Niger),...,Zinder,13.6500,9.1833,1,Studio Kalangou; Actu (Niger),National,"On 1 October 2020, a number of university stud...",0,1601932935,NER
3,7239538,562,NIR1465,1465,2020-09-30,2020,1,Violence against civilians,Attack,Military Forces of Niger (2011-),...,Toumour,13.6669,13.1198,2,Whatsapp,New media,"On 30 September 2020, Nigerien soldiers report...",2,1601932936,NER
4,7239540,562,NIR1466,1466,2020-09-30,2020,1,Violence against civilians,Abduction/forced disappearance,Islamic State (West Africa),...,Zarwaram,13.1981,12.4835,1,Whatsapp,New media,"On 30 September 2020, ISWAP militants reported...",0,1601932936,NER


### Do the right thing, take some time to look at the codebook and see what these columns are

In [8]:
niger_res.columns


Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

### Homework --- Make a map of some ACLED Data (absolutely use the code from the Global Terrorism Database exercise)

In [12]:
# Import additional libraries

from bokeh.plotting import figure, output_notebook, show #builds interactive graphs for python
from bokeh.models import Range1d
import math #this is used in graphic section to use the irrational number pi
output_notebook() #Allows inline plotting for Juptyer notebook

#Imports necessary aspects of Bokeh for plotting on a map
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer
tile_provider = get_provider('STAMEN_TERRAIN')

Loading BokehJS ...

In [11]:
# niger_res.info()